In [1]:
import torch

from pack_and_unpack import pack_scalar_index, unpack_scalar_index, pack_index, unpack_index_tensor

In [2]:
matrix = torch.randint(low=-4, high=4, size=(64, 64, 3, 3), dtype=torch.int8)
matrix

tensor([[[[ 3, -2,  2],
          [-2, -4, -1],
          [-4, -3,  2]],

         [[ 0, -2,  2],
          [ 0,  0,  3],
          [ 1, -4,  2]],

         [[-3,  1,  0],
          [ 1,  0,  3],
          [ 2,  3, -2]],

         ...,

         [[ 1,  3, -1],
          [-4, -2, -1],
          [ 1,  0, -3]],

         [[-3,  0, -2],
          [ 3,  1, -1],
          [-3, -1, -1]],

         [[-3, -4,  0],
          [ 1, -3, -3],
          [ 2,  2,  1]]],


        [[[-3, -1, -1],
          [-2,  3, -3],
          [-3, -4,  2]],

         [[ 1, -4,  0],
          [-2,  3,  2],
          [ 1, -4, -4]],

         [[ 0,  2,  1],
          [ 2, -3, -1],
          [ 2,  0,  2]],

         ...,

         [[-2, -2,  2],
          [-2,  3,  0],
          [ 2,  0, -2]],

         [[ 3, -3, -1],
          [-2, -4,  2],
          [-2, -2, -3]],

         [[-2, -2,  2],
          [ 0, -3,  1],
          [-2,  3, -3]]],


        [[[-2, -2,  3],
          [-4,  2, -2],
          [-2, -1, -4]],

    

In [3]:
packed, tensor_shape, tensor_dtype = pack_scalar_index(tensor=matrix, bit_width=3)
packed.shape, tensor_shape

(torch.Size([3456]), [64, 64, 3, 3])

In [4]:
unpacked = unpack_scalar_index(
		packed=packed, 
		tensor_shape=tensor_shape,
		tensor_dtype=tensor_dtype,
		bit_width=3
	)

unpacked

tensor([[[[ 3, -2,  2],
          [-2, -4, -1],
          [-4, -3,  2]],

         [[ 0, -2,  2],
          [ 0,  0,  3],
          [ 1, -4,  2]],

         [[-3,  1,  0],
          [ 1,  0,  3],
          [ 2,  3, -2]],

         ...,

         [[ 1,  3, -1],
          [-4, -2, -1],
          [ 1,  0, -3]],

         [[-3,  0, -2],
          [ 3,  1, -1],
          [-3, -1, -1]],

         [[-3, -4,  0],
          [ 1, -3, -3],
          [ 2,  2,  1]]],


        [[[-3, -1, -1],
          [-2,  3, -3],
          [-3, -4,  2]],

         [[ 1, -4,  0],
          [-2,  3,  2],
          [ 1, -4, -4]],

         [[ 0,  2,  1],
          [ 2, -3, -1],
          [ 2,  0,  2]],

         ...,

         [[-2, -2,  2],
          [-2,  3,  0],
          [ 2,  0, -2]],

         [[ 3, -3, -1],
          [-2, -4,  2],
          [-2, -2, -3]],

         [[-2, -2,  2],
          [ 0, -3,  1],
          [-2,  3, -3]]],


        [[[-2, -2,  3],
          [-4,  2, -2],
          [-2, -1, -4]],

    

In [11]:
indices = torch.tensor([0, 1, 2, 6, 7], dtype=torch.uint8)
print(indices)

index_bits = 3

wf = torch.arange(0, index_bits).to(indices.device)#.view(1, 1, 1, -1)
out = torch.bitwise_right_shift(indices.unsqueeze(-1), wf)
print(out, out.dtype)
#print("out:", out, out.shape, out.dtype)
out = torch.bitwise_and(out, 1, out=out)
print(out, out.dtype)
print(indices.shape[:-1])
out = out.reshape(*indices.shape[:-1], -1)
print("out:", out, out.shape, out.dtype)

paded_bits = (
        32 - out.reshape(*indices.shape[:-1], -1).shape[-1] % 32
    ) % 32
print("paded_bits:", paded_bits)

out = torch.nn.functional.pad(
        out,
        (0, paded_bits),
        value=0,
        mode="constant",
    ).reshape(*indices.shape[:-1], -1, 32)
print("out:", out, out.shape, out.dtype)

wf1 = torch.arange(0, 32, 1).to(indices.device)#.view(1, 1, 1, -1)
print(wf1)

out = torch.bitwise_left_shift(out, wf1)
print("out:", out, out.shape, out.dtype)

out = out.sum(dim=-1).to(torch.uint32).view(torch.int32)
print("out:", out, out.shape, out.dtype)

tensor([0, 1, 2, 6, 7], dtype=torch.uint8)
tensor([[0, 0, 0],
        [1, 0, 0],
        [2, 1, 0],
        [6, 3, 1],
        [7, 3, 1]]) torch.int64
tensor([[0, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 1, 1],
        [1, 1, 1]]) torch.int64
torch.Size([])
out: tensor([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1]) torch.Size([15]) torch.int64
paded_bits: 17
out: tensor([[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]) torch.Size([1, 32]) torch.int64
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])
out: tensor([[    0,     0,     0,     8,     0,     0,     0,   128,     0,     0,
          1024,  2048,  4096,  8192, 16384,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]]) torch.Size([1, 32]) torch.int64
out: tensor([31880], dtype

In [2]:
indice = torch.randint(low=0, high=7, size=(100,), dtype=torch.int8)
print(indice)

tensor([6, 6, 5, 5, 2, 1, 2, 4, 2, 3, 1, 5, 6, 3, 3, 5, 5, 4, 2, 4, 6, 4, 2, 5,
        1, 2, 0, 6, 0, 0, 1, 6, 2, 5, 6, 5, 3, 2, 2, 2, 4, 3, 3, 1, 4, 0, 6, 2,
        4, 1, 4, 1, 0, 1, 0, 1, 2, 5, 0, 1, 6, 1, 5, 3, 1, 0, 1, 5, 2, 6, 2, 1,
        4, 6, 1, 0, 5, 5, 3, 5, 6, 6, 5, 3, 0, 4, 2, 4, 1, 1, 1, 2, 6, 4, 2, 1,
        4, 4, 5, 4], dtype=torch.int8)


In [3]:
packed = pack_index(
    indice=indice,
    index_bits=3,
    index_dtype=torch.int8,
)

In [4]:
unpacked_indices = unpack_index_tensor(
    packed_tensor=packed,
    index_bits=3,
    num_elements=100
)
unpacked_indices = unpacked_indices.to(torch.int8)

In [5]:
print(indice.dtype, indice.shape)
print(packed.dtype, packed.shape)
print(unpacked_indices.dtype, unpacked_indices.shape)

torch.int8 torch.Size([100])
torch.int32 torch.Size([10])
torch.int8 torch.Size([100])


In [8]:
print(indice)
print(unpacked_indices)

tensor([6, 6, 5, 5, 2, 1, 2, 4, 2, 3, 1, 5, 6, 3, 3, 5, 5, 4, 2, 4, 6, 4, 2, 5,
        1, 2, 0, 6, 0, 0, 1, 6, 2, 5, 6, 5, 3, 2, 2, 2, 4, 3, 3, 1, 4, 0, 6, 2,
        4, 1, 4, 1, 0, 1, 0, 1, 2, 5, 0, 1, 6, 1, 5, 3, 1, 0, 1, 5, 2, 6, 2, 1,
        4, 6, 1, 0, 5, 5, 3, 5, 6, 6, 5, 3, 0, 4, 2, 4, 1, 1, 1, 2, 6, 4, 2, 1,
        4, 4, 5, 4], dtype=torch.int8)
tensor([6, 6, 5, 5, 2, 1, 2, 4, 2, 3, 1, 5, 6, 3, 3, 5, 5, 4, 2, 4, 6, 4, 2, 5,
        1, 2, 0, 6, 0, 0, 1, 6, 2, 5, 6, 5, 3, 2, 2, 2, 4, 3, 3, 1, 4, 0, 6, 2,
        4, 1, 4, 1, 0, 1, 0, 1, 2, 5, 0, 1, 6, 1, 5, 3, 1, 0, 1, 5, 2, 6, 2, 1,
        4, 6, 1, 0, 5, 5, 3, 5, 6, 6, 5, 3, 0, 4, 2, 4, 1, 1, 1, 2, 6, 4, 2, 1,
        4, 4, 5, 4], dtype=torch.int8)
